In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np

In [2]:
from pathlib import Path

p = Path(r"C:\Users\TSJ20\Desktop")
print(p.exists(), p)
print([f.name for f in p.glob("CTG*")])

True C:\Users\TSJ20\Desktop
['CTG(1).xls']


In [3]:
%pip install xlrd

Note: you may need to restart the kernel to use updated packages.


In [4]:
path = r"C:\Users\TSJ20\Desktop\CTG(1).xls"
df = pd.read_excel(path, engine="xlrd")
df.head()

,Unnamed: 0,Unnamed: 1,Cardiotocographic data,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,NaN,NaN,2126 measurements and classifications of foeta...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Worksheet,NaN,Raw Data,NaN,NaN,NaN,NaN,NaN,NaN,Data (temporal features per unit time),NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,K-W,NaN,DR is removed since p(K-W)=1
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Features,NaN,chi-square,p,NaN


In [5]:
raw = pd.read_excel(path, sheet_name=0, header=None)
header_row = raw.notna().sum(axis=1).idxmax()
df = pd.read_excel(path, sheet_name=0, header=header_row)
df = df.dropna(how='all').dropna(axis=1, how='all')
df = df.loc[:, ~df.columns.astype(str).str.contains("^Unnamed")]
df.head()

,Exam data,FileName,of CTG examination,LB,243.5,0
0,NaN,Date,of the examination,AC,1701.2,0.0
1,NaN,b,start instant,FM,84.3,0.0
2,NaN,e,end instant,UC,253.8,0.0
3,Measurements,LBE,baseline value (medical expert),DL,1336.1,0.0
4,NaN,LB,baseline value (SisPorto),DS,96.4,0.0


In [6]:
target_col = 'NSP' if 'NSP' in df.columns else 'CLASS'
df_num = df.select_dtypes(include=[np.number]).dropna(how='all')

In [7]:
p_train  = r"../data/manipulated_data_set/train_data.csv"
p_test   = r"../data/manipulated_data_set/test_data.csv"

In [8]:
train = pd.read_csv(p_train)
test  = pd.read_csv(p_test)

In [9]:
train.columns = train.columns.astype(str).str.strip()
test.columns  = test.columns.astype(str).str.strip()
target = 'NSP' if 'NSP' in train.columns else ('CLASS' if 'CLASS' in train.columns else None)

In [36]:
candidate_leaks = ['NSP','CLASS','FileName','SegFile','Date','b','d','e', target]
leak_cols = [c for c in candidate_leaks if c in train.columns] 

In [38]:
if target is None:
    print('?')

In [40]:
X_train = train.drop(columns=leak_cols)
y_train = train[target]
X_test  = test.drop(columns=leak_cols)
y_test  = test[target]

In [46]:
clf = DecisionTreeClassifier(
    random_state=42,
    class_weight='balanced',  
    max_depth=7,             
    min_samples_leaf=5
)
clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight='balanced', max_depth=7, min_samples_leaf=5,
                       random_state=42)

In [48]:
pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, pred))
print(classification_report(y_test, pred))
print("Confusion matrix:\n", confusion_matrix(y_test, pred))

Accuracy: 0.8934169278996865
              precision    recall  f1-score   support

         1.0       0.98      0.88      0.93       496
         2.0       0.69      0.91      0.79       101
         3.0       0.67      1.00      0.80        41

    accuracy                           0.89       638
   macro avg       0.78      0.93      0.84       638
weighted avg       0.92      0.89      0.90       638

Confusion matrix:
 [[437  41  18]
 [  7  92   2]
 [  0   0  41]]


In [50]:
fi = pd.Series(clf.feature_importances_, index=X_train.columns).sort_values(ascending=False)
print("Top features:\n", fi.head(10))

Top features:
 Mean      0.252500
MSTV      0.233082
ALTV      0.134926
ASTV      0.127828
AC        0.073964
Median    0.059191
DP        0.038090
Min       0.025120
Max       0.018288
Mode      0.013501
dtype: float64
